# Manual Review of Promising Candidates

Here we list out a bunch of candidates and their parameters which have promising weirdness in their lightcurves...
We'll add that to our talk... once the data is cumulated

We're rewiring this module to make it more automatic and generic so that we can utilize it better for the alien hunt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import pandas as pd
from scipy.optimize import curve_fit
import aliensims as dy
from ellc import lc
import os
from transit import occultnonlin, occultquad
from interpolation import load_grids, lc_interpolate_v2

### List of candidates
We're gonna do a generic search now and do away with this list
<ol>
    <li>KIC 8110757: potential eclipsing binary but highly assymetric curve
    <li>KIC 5357901 (KOI 188b, Kepler-425): tidally fluctuating hot jupiter
    <li>KIC 5358624: tidally fluctuating hot jupiter
    <li>KIC 9941662: seemsto be contact binary signatures
    <li>KIC 6372268: contact binary signatures
    <li>KIC 9025971: seems to be some tiny mid transit flares... could just be noise
    <li>KIC 8308347: personal curiosity
    <li>KIC 10975146: assymetric like 1 ... confirmed planet
    <li>KIC 2557430: assymetric peaks
    <li>KIC 10295951: some peaky sigs ... still candidate
    <li>KIC 11404644: Probably false alarm
    <li>KIC 6620003: shows some improvement
    <li>KIC 8323764:
</ol>

[8840] CAN rank122_009944201_1.jpg
[721] CAN rank131_008616873_1.jpg
[393] CAN rank237_001161345_1.jpg
[216] CAN rank272_007380537_1.jpg
[956] CAN rank306_006620003_1.jpg
[7855] CAN rank368_006842345_1.jpg
[582] CAN rank377_010295951_1.jpg
[7596] CAN rank384_011404644_1.jpg
        [3044] CAN rank386_010395543_1.jpg
[5676] CAN rank402_009574614_1.jpg
        [1531] CAN rank434_010158418_1.jpg
        [92] CAN rank446_003453214_1.jpg
            [3534] CAN rank488_006309763_1.jpg
     [37] CAN rank496_011138155_1.jpg
    [1204] CAN rank500_012120943_1.jpg
    [269] CAN rank508_009140402_1.jpg
    [2757] CAN rank517_008684730_1.jpg
      [3015] CAN rank518_009119458_1.jpg
[1646] CAN rank540_006300348_1.jpg
[2739] CAN rank552_007051180_1.jpg
     [3602] CAN rank582_002569995_1.jpg
      [172] CAN rank585_006526710_1.jpg
[5387] CAN rank586_008323764_1.jpg

In [ ]:
# opening the catalog and taking in the list of required values: Rorb, Rpl, Limb Parameters, impact parameter
catalog = ascii.read('../../../ex_TCE_sum.tab')

cat_kid = catalog['kepid']
cat_lab = catalog['koi_disposition'] 
cat_rorb = catalog['koi_dor']
cat_rpl = catalog['koi_ror']
cat_u1 = catalog['koi_ldm_coeff1']
cat_u2 = catalog['koi_ldm_coeff2']
cat_imp = catalog['koi_impact']
cat_pl_no = catalog['koi_tce_plnt_num']

In [ ]:
# we need a function that inputs the kepler id and planet number and gives us the name from the scandir
def filename_from_kepid(kepid,pl_no, directory):
    """ This function searches for the data from directory and values from catalog and returns the same

    :param kepid: the kepler id to search for
    :param pl_no: the TCE number within the kepler id
    :param directory: the list of elements in the directory obtained from os.listdir(filepath)
    
    returns - 
    filename, label, rorb, rpl, u1, u2, impact parameter

    """

    global cat_kid
    kepid = ('0000'+str(kepid))[-9:]
    out = np.char.find(directory, kepid)

    cat_kid = np.array([('0000'+str(el))[-9:] for el in cat_kid])
    loc = np.where(cat_kid == kepid)[0]
    locf = [el for el in loc if(cat_pl_no[el]==pl_no)]

    if(len(np.where(out>-1)[0])>0 and len(locf)>0): 
        return(directory[np.where(out>-1)[0][0]], cat_lab[locf[0]], cat_rorb[locf[0]], cat_rpl[locf[0]], cat_u1[locf[0]], cat_u2[locf[0]], cat_imp[locf[0]])
    else: return('none',0,0,0,0,0,0)

def sort_data(hdu, hduno):
    """ gives us a phase-folded, sorted, centered LC from a fits file 

    :param hdu: obtained from fits.open(file)
    :param hduno: the TCE number in the file
    
    returns - 
    a pandas dataframe containing details of stuff,
    transit period(in days),
    transit duration(in days),
    noise, in the form of standard deviation

    """
    
    flux = []
    flux_white=[]
    phase = []
    lc_err=[]
    model = []
    try: 
        tp = hdu[hduno].header['TPERIOD']
        td = hdu[hduno].header['TDUR']/24
    except: return(0,0,0,0)
    for ph, fl,wl, ml, err in zip(hdu[hduno].data['PHASE'],hdu[hduno].data['LC_DETREND'],hdu[hduno].data['LC_WHITE'],hdu[hduno].data['MODEL_WHITE'],hdu[hduno].data['LC_INIT_ERR']):
        if not np.isnan(fl):
            flux.append(fl)
            if(ph*2/tp>1): phase.append(ph*2/tp -2)
            else: phase.append(ph*2/tp)
            flux_white.append(wl)
            model.append(ml)
            lc_err.append(err)
    dfunb = pd.DataFrame(list(zip(phase, flux, flux_white, model, lc_err)),columns=['phase', 'flux','flux_white', 'model','error'])
    df=dfunb.sort_values('phase',axis=0,ascending=True)
    df=df[(df.phase>-0.75) & (df.phase<0.75)]

    # bins=np.linspace(min(df['phase']),max(df['phase']),8000)
    # groups = df.groupby(np.digitize(df['phase'], bins))
    # df=groups.median()
    
    # print('size:',len(df)) 

    df_noise = df[(df.phase<-3*td/tp) | (df.phase>3*td/tp)]
    noise = np.std(np.array(df_noise['flux']))
    return(df, hdu[hduno].header['TPERIOD'], hdu[hduno].header['TDUR']/24, noise)


In [ ]:
#Directory setting for candidate list
#FILEPATH_FPS="E:\Masters_Project_Data\\alienworlds_fps\\"
FILEPATH_FPS="/media/ushasi/Elements/Masters_Project_Data/alienworlds_fps/"
entries = os.listdir(FILEPATH_FPS)

k1 = 'kplr008110757-20160128150956_dvt.fits'
k2 = 'kplr005357901-20160128150956_dvt.fits'
k3 = 'kplr005358624-20160128150956_dvt.fits'
k4 = 'kplr009941662-20160128150956_dvt.fits'
k5 = 'kplr006372268-20160128150956_dvt.fits'
k6= 'kplr009025971-20160128150956_dvt.fits'
k7= 'kplr008308347-20160128150956_dvt.fits'
k8= 'kplr010975146-20160128150956_dvt.fits'
k9='kplr002557430-20160128150956_dvt.fits'
k10='kplr010295951-20160128150956_dvt.fits'
k11='kplr011404644-20160128150956_dvt.fits'
k12='kplr006620003-20160128150956_dvt.fits'
k13='kplr008323764-20160128150956_dvt.fits'

ks1='kplr001161345-20160128150956_dvt.fits'
ks2='kplr009944201-20160128150956_dvt.fits'
ks3='kplr009574614-20160128150956_dvt.fits'
ks4='kplr006300348-20160128150956_dvt.fits'
ks5='kplr007051180-20160128150956_dvt.fits'

kt1='kplr006681618-20160128150956_dvt.fits'
kt2='kplr008240109-20160128150956_dvt.fits'
kt3='kplr009597095-20160128150956_dvt.fits'

points, vals, phases = load_grids()



### Model fitting functions
Here are routines needed for fitting the models if and when...

In [ ]:
#mandel and agol
def new_plar_ma(ph,p,u1,u2,rorb,imp):
    znp = np.sqrt(np.abs(rorb*np.sin(ph*np.pi))**2+imp**2)
    a= occultquad(znp,p,[u1,u2])  
    return(a -1) 

#2d interpolate model (self made)
def new_plar_2d(ph,p,u1,u2,rorb,imp):
    a= lc_interpolate_v2(ph*np.pi, p, rorb, imp, u1,u2,points, vals, phases) 
    return(a -1) 
    
#ellc model
def new_plar_ellc(ph,p,u1,u2,rorb, imp):
    incl = np.arccos(imp/rorb)*180/np.pi
    # a= lc(ph,radius_1=1/rorb,radius_2=p/rorb,sbratio=0.0,incl=incl,ld_1='quad',ldc_1=[u1,u2],
    #     shape_1='sphere', shape_2='sphere',f_c=np.sqrt(e)*np.cos(w),f_s=np.sqrt(e)*np.sin(w))
    a= lc(ph,radius_1=1/rorb,radius_2=p/rorb,sbratio=0.0,incl=incl,ld_1='claret',ldc_1=[u1,u2,0,0],
         shape_1='sphere', shape_2='sphere', period=2)
    return(a -1) 

#ellc model with a tidal parameter
def new_plar_ellc_tide(ph,p,u1,u2,rorb, imp, hf):
    incl = np.arccos(imp/rorb)*180/np.pi
    # a= lc(ph,radius_1=1/rorb,radius_2=p/rorb,sbratio=0.0,incl=incl,ld_1='quad',ldc_1=[u1,u2],
    #     shape_1='sphere', shape_2='sphere',f_c=np.sqrt(e)*np.cos(w),f_s=np.sqrt(e)*np.sin(w))
    a= lc(ph,radius_1=1/rorb,radius_2=p/rorb,sbratio=0.0,incl=incl,ld_1='quad',ldc_1=[u1,u2],
         shape_1='sphere', shape_2='love', hf_2=hf, hf_1=hf, period=2)
    return(a -1)    

Opening file and header:

In [ ]:
hdu = fits.open(FILEPATH_FPS+kt3)
print("number of TCEs:", len(hdu)-2)

df, tp, td = sort_data(hdu,1)

df_noise = df[(df.phase<-3*td/tp) | (df.phase>3*td/tp)]
noise = np.std(np.array(df_noise['flux']))
print(noise)
print(td, tp)
#print(hdu[0].header)

In [ ]:
plt.plot(hdu[1].data['PHASE'],hdu[1].data['LC_DETREND'], marker='.',ls='None')
print(hdu[1].data['LC_INIT_ERR'])
master_u1=0.2412
master_u2=0.3287
#plt.xlim(-10,10)


### Limb Darkening Statistics
<table>
<tr>
    <td>KIC</td>
    <td>Rpl
    <td>Rorb
    <td>u1
    <td>u2
    <td>b
    <td>e
</tr>
<tr>
    <td>KIC 6620003</td>
    <td>0.16913
    <td>5.229
    <td>0.3803
    <td>0.3384
    <td>1.182
    <td>0
</tr>
<tr>
    <td>KIC 8323764</td>
    <td>0.16913
    <td>15.85
    <td>0.49
    <td>0.2075
    <td>0.557
    <td>0
</tr>
<tr>
    <td>KIC 11404644</td>
    <td>0.2159
    <td>9.7742
    <td>0.3147
    <td>0.3032
    <td>0.168
    <td>0
</tr>
<tr>
    <td>KIC 10295951</td>
    <td>0.114797
    <td>6.291
    <td>0.2412
    <td>0.3287
    <td>0.57
    <td>0
</tr>
<tr>
    <td>KIC 2557430</td>
    <td>0.43452 
    <td>1.93 
    <td>0.2893
    <td>0.3105
    <td>1.172 
    <td>0
</tr>
<tr>
    <td>KIC 8110757</td>
    <td>0.31774 (0.037)
    <td>1.528 (3.5193)
    <td>0.3975 
    <td>0.2650
    <td>1.262 (34.34 deg)
    <td>0
</tr>
<tr>
    <td>KIC 5357901</td>
    <td>0.108759
    <td>13.635
    <td>0.5256
    <td>0.1860
    <td>0.602 (88.65 deg)
    <td>0
</tr>
<tr>
    <td>KIC 5358624</td>
    <td>0.13372
    <td>11.699
    <td>0.5167
    <td>0.1914
    <td>0.12 (89.41 deg)
    <td>0
</tr>
<tr>
    <td>KIC 9651668</td>
    <td>0.121923
    <td>8.546
    <td>0.4415
    <td>0.2396
    <td>0.076 (89.49 deg)
    <td>0
</tr>
<tr>
    <td>KIC 5596440</td>
    <td>0.04377
    <td>1.346
    <td>0.2913
    <td>0.3120
    <td>0.972 (43.72 deg)
    <td>0
</tr>
<tr>
    <td>KIC 6372268</td>
    <td>0.04,0.03,0.002
    <td>24.4,17.7,51.52
    <td>1,-0.29,0.5 (-.1)
    <td>0,1,0
    <td>0
    <td>0
</tr>
</table>

### Fits and Parameters

In [ ]:

popt1, pcov1 = curve_fit(new_plar_ma, np.array(df['phase']), np.array(df['flux']), 
    bounds=([0,master_u1-0.01,master_u2-0.01,1,0], [1,master_u1+0.01,master_u2+0.01,40,1]))

print("Var3d",np.trace(pcov1))
print("Params3d:",np.round(popt1,4))
print('rchi3d:',np.mean((df['flux']-new_plar_ma(np.array(df['phase']), *popt1))**2)/noise**2)

In [ ]:
popt3, pcov3 = curve_fit(new_plar_ellc_tide, np.array(df['phase']), np.array(df['flux']), 
    bounds=([0,master_u1-0.01,master_u2-0.01,1,0,1.5], [1,master_u1+0.01,master_u2+0.01,40,1.2,4.5]))

print("Var3d",np.trace(pcov3))
print("Params3d:",np.round(popt3,4))
print('rchi3d:',np.mean((df['flux']-new_plar_ellc_tide(np.array(df['phase']), *popt3))**2)/noise**2)

In [ ]:
popt2, pcov2 = curve_fit(new_plar_2d, np.array(df['phase']), np.array(df['flux']), 
    bounds=([0,master_u1-0.01,master_u2-0.01,1,0.9], [1,master_u1+0.01,master_u2+0.01,40,1.3]))

print("Var2d",np.trace(pcov2))
print("Params2d:",np.round(popt2,4))
print('rchi2d:',np.mean((df['flux']-new_plar_2d(np.array(df['phase']), *popt2))**2)/noise**2)

In [ ]:
from scipy.interpolate import interp1d
dfa=pd.read_csv('811_fit.csv')
check = np.array(dfa['model_fit'])

### Plotting the fits

In [ ]:
ca=["#e27c7c", "#a86464", "#6d4b4b", "#503f3f", "#333333", "#3c4e4b", "#466964", "#599e94", "#6cd4c5"]
fig = plt.figure(figsize=(10,12))
kernel =np.ones(5)/5

print(len(np.array(df['flux'])))
fl=np.array(df['flux'])
ph=np.array(df['phase'])
#noise=np.array(df['error'])
ax1 = plt.subplot2grid((3,1), (0, 0), rowspan=1)
ax2 = plt.subplot2grid((3,1), (1, 0), rowspan=2)
#ax3 = plt.subplot2grid((3,1), (3, 0), rowspan=1)

exp1 = new_plar_ma(ph, *popt1)

rchi3d=np.sum((fl[900:1500]-exp1[900:1500])**2/noise**2)/(len(fl[900:1500])-3)
print('rchi3d_ma:',rchi3d)
ax1.plot(ph, exp1, label='3d', color=ca[3])
ax2.plot(ph,np.convolve(fl-exp1,  kernel, mode='same')+0.00, label='3d:$\chi^2$'+str(np.around(rchi3d,4)), color=ca[3])


exp2 = new_plar_2d(ph, *popt2)
exp2=check
rchi2d=np.sum((fl[900:1500]-exp2[900:1500])**2/noise**2)/(len(fl[900:1500])-3)
print('rchi2d:',rchi2d)
ax1.plot(ph,exp2, label='2d', color=ca[5])
ax2.plot(ph,np.convolve(fl-exp2,  kernel, mode='same')-0.002, label='2d:$\chi^2$'+str(np.around(rchi2d,4)), color=ca[5])

# exp3 = new_plar_ellc_tide(ph, *popt3)
# rchi3d_el=np.sum((fl[900:1500]-exp3[900:1500])**2/noise[900:1500]**2)/(len(fl[900:1500])-3)
# print('rchi3d_el:',rchi3d_el)
# ax1.plot(ph, exp3, label='3d(tide)', color=ca[7])
# ax2.plot(df['phase'],np.convolve(fl-exp3,  kernel, mode='same')-0.002, label='3d(tide):$\chi^2$'+str(np.around(rchi3d_el,4)), color=ca[7])

expc = new_plar_ma(ph, 0.3177, master_u1, master_u2, 1.528, 1.262)
rchipub=np.sum((fl[900:1500]-expc[900:1500])**2/noise**2)/(len(fl[900:1500])-3)
print('rchi3d_c:',rchipub)
ax1.plot(ph, expc,  label='pub', color=ca[1])
ax2.plot(ph,np.convolve(fl-expc,  kernel, mode='same')+0.002, label='pub:$\chi^2$'+str(np.around(rchipub,4)),color=ca[1])

ax1.set_title('Model Fit: 8110757\nMandel and Algol 2002', size=17)
ax1.plot(ph,fl, color='gray', zorder=1, marker='.', ls='None')

ax2.set_title('Residuals', size=14)
ax2.set_xlabel('Phase', size=14)
ax1.set_ylabel('Flux', size=14)
ax1.legend()
ax2.set_ylabel('Flux-Model', size=14)
ax1.set_xlim(-0.4,0.4)
ax2.set_xlim(-0.4,0.4)
ax3.set_xlim(-0.4,0.4)

# ax3.plot(ph,(exp1-exp2)*10**6, color=ca[3])
# ax3.set_ylabel('3d - 2d (ppm)',size=14)

ax2.legend(loc='upper left')

#print('old',np.std(exp- df['flux']), 'new:',np.std(exp2- df['flux']))
plt.savefig("fprez_8110757_2.png")


In [ ]:
plt.plot(exp1-exp2)

### Bulk Codes

This is to do the same things as we did above, but in  bulk, so we encapsulate all this into some functions

In [ ]:
def bulk_fitting(df, u1, u2, noise):
    """ cumulating all the fitting parameters together

    :param df: dataframe obtained from function 'sort data'
    :param u1: quadratic limb darkening coefficient
    :param u2: quadratic limb darkening coefficient

    returns:  - 
    fitted parameters for 3D fit(new_plar_ma), rchi of fit, 2D fit(new_plar_2d), rchi of fit

    """
    try: 
        popt1, pcov1 = curve_fit(new_plar_ma, np.array(df['phase']), np.array(df['flux']), 
        bounds=([0,u1-0.01,u2-0.01,1,0], [1,u1+0.01,u2+0.01,40,1.5]))
        rchi3d = np.mean((df['flux']-new_plar_ma(np.array(df['phase']), *popt1))**2)/noise**2
    except: 
        rchi3d=0
        popt1=[0,0,0,0,0]
    # print("Var3d:",np.trace(pcov1))
    # print("Params3d:",np.round(popt1,4))
    # print('rchi3d:',rchi3d)

    try: 
        popt2, pcov2 = curve_fit(new_plar_2d, np.array(df['phase']), np.array(df['flux']), 
        bounds=([0,u1-0.01,u2-0.01,1,0], [1,u1+0.001,u2+0.01,40,1.5]))
        rchi2d = np.mean((df['flux']-new_plar_2d(np.array(df['phase']), *popt2))**2)/noise**2

    except:
        rchi2d=0
        popt2=[0,0,0,0,0]

    # print("Var2d:",np.trace(pcov2))
    # print("Params2d:",np.round(popt2,4))
    # print('rchi2d:',rchi2d)

    # popt3, pcov3 = curve_fit(new_plar_ellc_tide, np.array(df['phase']), np.array(df['flux']), 
    # bounds=([0,u1-0.01,u2-0.01,1,0,1.5], [1,u1+0.01, u2+0.01,40,1.5,4.5]))

    # print("Var3d tide:",np.trace(pcov3))
    # print("Params3d tide:",np.round(popt3,4))
    # print('rchi3d tide:',np.mean((df['flux']-new_plar_ellc_tide(np.array(df['phase']), *popt3))**2)/noise**2)

    return(popt1, rchi3d, popt2, rchi2d)


def bulk_plotting(df, kepid, plno, popt1, popt2, u1, u2, rorb, rpl, imp):
   
    ca=["#e27c7c", "#a86464", "#6d4b4b", "#503f3f", "#333333", "#3c4e4b", "#466964", "#599e94", "#6cd4c5"]
    fig = plt.figure(figsize=(10,12))
    kernel =np.ones(5)/5

    fl=np.array(df['flux'])
    ph=np.array(df['phase'])
    #noise=np.array(df['error'])
    ax1 = plt.subplot2grid((4,2), (0, 0), rowspan=1, colspan=2)
    ax2 = plt.subplot2grid((4,2), (1, 0), rowspan=2, colspan=2)
    ax3 = plt.subplot2grid((4,2), (3, 0), rowspan=1, colspan=1)
    ax4 = plt.subplot2grid((4,2), (3, 1), rowspan=1, colspan=1)

    exp1 = new_plar_ma(ph, *popt1)
    exp2 = new_plar_2d(ph, *popt2)
    # exp3 = new_plar_ellc_tide(ph, *popt3)
    expc = new_plar_ma(ph, rpl, u1, u2, rorb, imp)

    plot_diff=0.5*max([max(exp1) - min(exp1),max(exp2) - min(exp2),max(expc) - min(expc)])

    rchi3d=np.sum((fl-exp1)**2/noise**2)/(len(fl)-3)
    ax1.plot(ph, exp1, label='3d', color=ca[3])
    ax2.plot(ph,np.convolve(fl-exp1,  kernel, mode='same'), label='3d:$\chi^2$'+str(np.around(rchi3d,4)), color=ca[3])

    # exp2=check
    rchi2d=np.sum((fl-exp2)**2/noise**2)/(len(fl)-3)
    ax1.plot(ph,exp2, label='2d', color=ca[5])
    ax2.plot(ph,np.convolve(fl-exp2,  kernel, mode='same')-plot_diff, label='2d:$\chi^2$'+str(np.around(rchi2d,4)), color=ca[5])

    
    # rchi3d_el=np.sum((fl-exp3)**2/noise**2)/(len(fl)-3)
    # print('rchi3d_el:',rchi3d_el)
    # ax1.plot(ph, exp3, label='3d(tide)', color=ca[7])
    # ax2.plot(df['phase'],np.convolve(fl-exp3,  kernel, mode='same')-plot_diff, label='3d(tide):$\chi^2$'+str(np.around(rchi3d_el,4)), color=ca[7])
    
    rchipub=np.sum((fl-expc)**2/noise**2)/(len(fl)-3)
    ax1.plot(ph, expc,  label='pub', color=ca[1])
    ax2.plot(ph,np.convolve(fl-expc,  kernel, mode='same')+plot_diff, label='pub:$\chi^2$'+str(np.around(rchipub,4)),color=ca[1])

    ax1.set_title('Model Fit: '+kepid+'\nMandel and Agol, ellc, and 2D', size=17)
    ax1.plot(ph,fl, color='gray', zorder=1, marker='.', ls='None')

    ax2.set_title('Residuals', size=14)
    ax2.set_xlabel('Phase', size=14)
    ax1.set_ylabel('Flux', size=14)
    ax1.legend()
    ax2.set_ylabel('Flux-Model', size=14)
    ax1.set_xlim(-0.4,0.4)
    ax2.set_xlim(-0.4,0.4)
    ax3.set_xlim(-0.4,0.4)

    ax3.plot(ph,(exp1-exp2)*10**6, color=ca[3])
    ax3.set_ylabel('3d - 2d (ppm)',size=14)
    ax3.set_xlabel('Phase')

    ax2.legend(loc='upper left')

    props = dict(boxstyle='round', facecolor=ca[8], alpha=0.5)
    textstr = "   Cat, 3D, 2D\n$R_{pl}$: "+str(np.around(rpl,3))+", "+str(np.around(popt1[0],3))+", "+str(np.around(popt2[0],3))+"\n$R_{orb}$: "+str(np.around(rorb,3))+", "+str(np.around(popt1[3],3))+", "+str(np.around(popt2[3],3))+"\n$b$: "+str(np.around(imp,3))+", "+str(np.around(popt1[4],3))+", "+str(np.around(popt2[4],3))+"\n$u_1$: "+str(np.around(u1,3))+", "+str(np.around(popt1[1],3))+", "+str(np.around(popt2[1],3))+"\n$u_2$: "+str(np.around(u2,3))+", "+str(np.around(popt1[2],3))+", "+str(np.around(popt2[2],3))
   
    ax4.text(0.1, 0.5, textstr, transform=ax4.transAxes, fontsize=12, verticalalignment='center', bbox=props)

    plt.tight_layout()
    #print('old',np.std(exp- df['flux']), 'new:',np.std(exp2- df['flux']))
    

In [ ]:
#one test example here
ename, elab,  erorb, erpl, eu1, eu2, eimp =  filename_from_kepid('6620003',1,entries)
hdu = fits.open(FILEPATH_FPS+ename)
print("number of TCEs:", len(hdu)-2)

df, tp, td, noise = sort_data(hdu,1)

fit1, rchi1, fit2, rchi2 = bulk_fitting(df,eu1, eu2, noise)

bulk_plotting(df, '6620003', 1, fit1, fit2, eu1, eu2, erorb, erpl, eimp)



In [ ]:
#bulk run code
import warnings
warnings.filterwarnings("ignore")
stI = 3000
endI = 3500

range_error_list=[]
for label,kid, plno in zip(cat_lab[stI:endI], cat_kid[stI:endI], cat_pl_no[stI:endI]):
    if(label!='CANDIDATE'): continue
    ename, elab,  erorb, erpl, eu1, eu2, eimp =  filename_from_kepid(kid,plno,entries)
    try: hdu = fits.open(FILEPATH_FPS+ename)
    except: continue
    df, tp, td, noise = sort_data(hdu,plno)
    if(tp==0 and td==0 and noise==0): continue
    if(erorb>40):
        print('range error:', kid, plno)
        range_error_list.append([kid,plno])
        continue
    
    fit1, rchi1, fit2, rchi2 = bulk_fitting(df,eu1, eu2, noise)
    if(rchi1==0 or rchi2==0):
        print('some error:', kid, plno)
        range_error_list.append([kid,plno])
        continue
    print(kid, plno, rchi1, rchi2)
    bulk_plotting(df, kid, plno, fit1, fit2, eu1, eu2, erorb, erpl, eimp)
    if(rchi1>rchi2): plt.savefig("../../fit_directory/came_right/fit_"+kid+"_"+str(plno)+".png")
    else: plt.savefig("../../fit_directory/came_wrong/fit_"+kid+"_"+str(plno)+".png")
    plt.close()

dfsave = pd.DataFrame(range_error_list,columns=['id','pl_no'])
dfsave.to_csv('../../fit_directory/range_error4_1.csv',index=False)

    

In [ ]:
# dfsave = pd.DataFrame(range_error_list,columns=['id','pl_no'])
# dfsave.to_csv('../../fit_directory/range_error2.csv',index=False)

In [ ]:
# np.where(cat_kid=='006119141')
print((cat_lab[1694:1700]=='CANDIDATE').sum())

## check out 1700 - 1800 range....some shit went down in it

### Demonstration of Weirdness

In [ ]:
ca=["#e27c7c", "#a86464", "#6d4b4b", "#503f3f", "#333333", "#3c4e4b", "#466964", "#599e94", "#6cd4c5"]

plt.style.use('seaborn-bright')
fig = plt.figure(figsize=(10,8))
ax1 = plt.subplot2grid((2,2), (0, 0), rowspan=1, colspan=2)
ax2 = plt.subplot2grid((2,2), (1,0), colspan=1)
ax3 = plt.subplot2grid((2,2), (1,1), rowspan=1)

ax1.plot(hdu[1].data['PHASE'],hdu[1].data['LC_DETREND'],color=ca[7],label='flux',marker='.',ls='None')
#ax1.plot(df['phase'],df['model'],color=ca[5],label='model')
ax2.plot(df['phase'],df['flux'],color=ca[7], marker='.',ls='None')
ax2.plot(df['phase'],new_plar_ma(df['phase'],*popt1),color=ca[5])
ax3.plot(df['phase'], df['flux']-new_plar_ma(df['phase'],*popt1),color=ca[4])

ax2.set_xlim(-0.3,0.3)
ax3.set_xlim(-0.3,0.3)

ax1.legend()
ax1.set_ylabel('Flux', size=14)
ax1.set_xlabel('phase($\pi$)', size=14)
ax2.set_ylabel('Flux', size=14)
ax2.set_xlabel('phase($\pi$)', size=14)
#ax3.set_ylabel('Flux-Model', size=14)
ax3.set_xlabel('phase($\pi$)', size=14)
#ax1.set_title('KIC 5359701\n $R_{pl}=0.11R_{st}$, $R_{orb}=11.19R_{st}$, $u_1$ = 0.53, $u_2$ =0.15 , b=0.602', size=15)
#ax1.set_title('KIC 8110757\n $R_{pl}=0.037R_{st}$, $R_{orb}=3.51R_{st}$, $u_1$ = 0.397, $u_2$ =0.265 , b=0', size=15)
ax1.set_title('KIC 10295951', size=15)

plt.savefig('frpez_kic10295951.png')
plt.show()

### Cadence Problem

In [ ]:
alt = 30
ph = np.linspace(-0.125,0.125,300)
fl = new_plar(ph, 0.12,0.0,0.0,3,0.0)
fl_smooth = np.convolve(fl,np.ones(alt)/alt,mode='same')
# ph_smooth = [ph[i] for i in range(50,len(ph),100)]
# fl_smooth = [np.mean(fl[i:i+100]) for i in range(0,len(fl),100)]
fig, ax = plt.subplots(1,1,figsize=(7,5))
ax.set_title('Cadence Problem \n $R_{pl}=0.1 R_{st}$,$R_{orb}=3 R_{st}$')
ax.plot(ph,fl, label='sim', color='green')
#ax.plot(ph_smooth, fl_smooth, label='avg')
ax.plot(ph, fl_smooth, label='avg', color='red')
ax.legend()
#plt.savefig('fprez_cadence.jpg')
#plt.plot(ph, fl-fl_smooth)
print(-np.max(fl-fl_smooth)/np.min(fl))

In [ ]:
alt = 100
devarr=[]
timearr=np.linspace(10,90,9)
mptarr=[]

ca2=["#e27c7c", "#a86464", "#6d4b4b", "#503f3f", "#333333", "#3c4e4b", "#466964", "#599e94", "#6cd4c5"]

fig, ax = plt.subplots(1,1,figsize=(7,5))
i=0
for t in timearr:
    temp=[]
    ptarr=np.floor(np.linspace(t,10000,40))
    mptarr.append(ptarr)
    for pt in ptarr:
        ph = np.linspace(-0.125,0.125,int(pt))
        fl = new_plar(ph, 0.1,0.0,0.0,3,0.0)
        fl_smooth = np.convolve(fl,np.ones(int(t))/int(t),mode='same')
        val=-np.max(fl-fl_smooth)/np.min(fl)
        temp.append(val)
    devarr.append(temp)
    ax.plot(ptarr,temp, label=str(int(t)), color=ca2[i])
    i+=1


ax.set_xscale('log')
ax.set_title('Cadence Problem \n $R_{pl}=0.1 R_{st}$,$R_{orb}=3 R_{st}$')
ax.set_ylabel('percent deviation')
ax.set_xlabel('Points')
ax.legend()

plt.savefig('fprez_cad_demo2.jpg')

